In [1]:
import sys
import pickle

import joblib
import numpy as np
from keras.models import load_model

module_path = '/home/kinfi4/python/Propaganda-Analyzer/src/ETL'
if module_path not in sys.path:
    sys.path.append(module_path)

from services.domain.text_preprocessor import TextPreprocessor
from config.config import MAX_POST_LEN_IN_WORDS

# Load models

In [2]:
vectorizer_loaded = pickle.load(open('./trained-models/vectorizer.pk', 'rb'))
tokenizer = pickle.load(open('./trained-models/keras-tokenizer.pk', 'rb'))

svc = joblib.load(open('./trained-models/svc-news-type-prediction.sav', 'rb'))
knn = joblib.load(open('./trained-models/knn-news-type-prediction.sav', 'rb'))
nb = joblib.load(open('./trained-models/nb-news-type-prediction.sav', 'rb'))
tree = joblib.load(open('./trained-models/tree-news-type-prediction.sav', 'rb'))

cnn_model = load_model('./trained-models/cnn-news-type-prediction.h5')
lstm_model = load_model('./trained-models/lstm-news-type-prediction.h5')

preprocessor = TextPreprocessor(
    sklearn_vectorizer=vectorizer_loaded, 
    keras_tokenizer=tokenizer
)

In [3]:
def get_predicted_label(label_idx):
    labels_indexes = {0: 'Economical', 1: 'Political', 2: 'Shelling', 3: 'Humanitarian'}
    
    return labels_indexes[label_idx]

# Predict using ML models

In [4]:
test_texts = [
    'Российские банки отключили от swift',
    'оккупанты обстреляли школу, более 10 детей погибли',
    'макдональдс уходит с рынка России',
    'Финляндия вступает в НАТО',
    'Путин сказал, что победит в этой войне',
    'Солдат с азовстали эвакуировали',
    'США заморозили резервы центробанка России',
    'Позиции войск РФ подверглись страшному обстрегу, более 100 окупантов мертво',
    'После отступления росийских военных из Бучи там были найдены тела мирных жителей',
    'Пленных солдат унижали, и вообще страх как пытали',
    'Цель операции россии украине заключается не разделе страны части обеспечении гарантий собственной безопасности заявил песков интервью'
]

test_vectors = preprocessor.sklearn_vectorize_text(test_texts, make_preprocessing=True)

In [6]:
predicted_indexes_svc = svc.predict(test_vectors.toarray())
predicted_indexes_knn = knn.predict(test_vectors.toarray())
predicted_indexes_nb = nb.predict(test_vectors.toarray())
predicted_indexes_tree = tree.predict(test_vectors.toarray())

for idx_svc, idx_knn, idx_nb, idx_tree, text in zip(
    predicted_indexes_svc,
    predicted_indexes_knn,
    predicted_indexes_nb,
    predicted_indexes_tree,
    test_texts
):
    svc_label = get_predicted_label(idx_svc)
    knn_label = get_predicted_label(idx_knn)
    nb_label = get_predicted_label(idx_nb)
    tree_label = get_predicted_label(idx_tree)
    
    if svc_label == knn_label == nb_label:
        predicted_label = svc_label
    else:
        predicted_label = f'SVC={svc_label} | KNN={knn_label} | NB={nb_label}'
    
    print(f'{predicted_label} -- {text}')
    print('-' * 30)

Economical -- Российские банки отключили от swift
------------------------------
SVC=Shelling | KNN=Humanitarian | NB=Humanitarian -- оккупанты обстреляли школу, более 10 детей погибли
------------------------------
Economical -- макдональдс уходит с рынка России
------------------------------
Political -- Финляндия вступает в НАТО
------------------------------
SVC=Political | KNN=Humanitarian | NB=Political -- Путин сказал, что победит в этой войне
------------------------------
Humanitarian -- Солдат с азовстали эвакуировали
------------------------------
Economical -- США заморозили резервы центробанка России
------------------------------
Shelling -- Позиции войск РФ подверглись страшному обстрегу, более 100 окупантов мертво
------------------------------
Humanitarian -- После отступления росийских военных из Бучи там были найдены тела мирных жителей
------------------------------
SVC=Humanitarian | KNN=Humanitarian | NB=Shelling -- Пленных солдат унижали, и вообще страх как пытал

# Predict using NN models

In [10]:
text_sequences = preprocessor.keras_tokenize_and_pad_text(
    test_texts,
    make_preprocessing=True,
    max_words_number=MAX_POST_LEN_IN_WORDS,
    padding='pre',
    truncating='post',
)

In [11]:
predicted_indexes_cnn = cnn_model.predict(text_sequences)
predicted_indexes_lstm = lstm_model.predict(text_sequences)

for cnn_result_vector, lstm_result_vector, text in zip(
    predicted_indexes_cnn,
    predicted_indexes_lstm,
    test_texts
):
    cnn_label = get_predicted_label(cnn_result_vector.argmax())
    lstm_label = get_predicted_label(lstm_result_vector.argmax())
    
    if cnn_label == lstm_label:
        predicted_label = cnn_label
    else:
        cnn_prediction = f''
        predicted_label = f'CNN={cnn_label} | LSTM={lstm_label}'
    
    print(f'{predicted_label} -- {text}')
    print('-' * 30)

Economical -- Российские банки отключили от swift
------------------------------
Shelling -- оккупанты обстреляли школу, более 10 детей погибли
------------------------------
Economical -- макдональдс уходит с рынка России
------------------------------
Political -- Финляндия вступает в НАТО
------------------------------
CNN=Economical | LSTM=Political -- Путин сказал, что победит в этой войне
------------------------------
Humanitarian -- Солдат с азовстали эвакуировали
------------------------------
Economical -- США заморозили резервы центробанка России
------------------------------
CNN=Humanitarian | LSTM=Shelling -- Позиции войск РФ подверглись страшному обстрегу, более 100 окупантов мертво
------------------------------
CNN=Humanitarian | LSTM=Shelling -- После отступления росийских военных из Бучи там были найдены тела мирных жителей
------------------------------
CNN=Shelling | LSTM=Humanitarian -- Пленных солдат унижали, и вообще страх как пытали
----------------------------